In [1]:
# This version uses abstracted functions that actually work to load X and Y training data into arrays of the right shape
# data successfully flows through LSTM connected to a flatten and dense layer

# Here I will be building out the architecture of the first classification LSTM
# At each time step, this LSTM will take in a vector representing the extracted audio and visual features from Liris-ACCEDE
# Its goal is to output whether or not the movie induces fear at each time step

# First, import necessary libraries
import tensorflow as tf
import numpy as np

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local06 import *

Using TensorFlow backend.


In [3]:
# uploading the X values
X_input = load_Xinput(get_fc6_directory("07"))
print(X_input.shape)

(212, 4096)


In [4]:
# uploading the Y_values
# the target data (y_values) will be a one-hot vector representing which seconds of movie induce fear

# convert into function that takes following input args: movie_length, fear_annot_path
# returns y_data_input

# first access start and stop times for fear-inducing sequences
y_data = np.loadtxt('fear_annotations_part01/MEDIAEVAL18_07_Fear.txt', skiprows=1)

# now treat these as pairs of indices --> we want all the indices between each pair of numbers
# create array of zeros --> the size will be the number of seconds in movie, in this case 210
movie_length = 212 #MAGIC NUMBER ALERT! --> length of movie
y_data_input = np.zeros((movie_length)) 

# for each element in first dimension of the y_data array
for i in range(y_data.shape[0]):
    # access the start time number and end time number
    start = int(y_data[i][0])
    end = int(y_data[i][1])
    # set the elements between these indices in the zeros array to one
    y_data_input[start] = 1 #maybe superfluous
    y_data_input[end] = 1
    y_data_input[start:end] = 1
print(y_data_input.shape)

(212,)


In [11]:
# setting up some key values
timesteps = 212   # the number of seconds in movie 07 --> i will figure out how to programmatically get this value
data_dim = 4096    # the number of output values from VGG16 layer fc6 --> switch to programmatic later
# could data_dim be the number of features that have been extracted (for now visual features only) --> maybe too much

X_train_data = np.zeros([1, timesteps, data_dim])
Y_train_data = np.zeros([1, timesteps])

# I have yet to figure this out
X_train_data[0, :, :] = X_input # the new axis will eventually become the batch size
print(X_train_data.shape)
Y_train_data[0, :] = y_data_input
print(Y_train_data.shape)

batch_size = 1 # very much arbitrary --> batch size relates to number of movies being put in
num_epochs = 20 # very much arbitrary

(1, 212, 4096)
(1, 212)


In [18]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()
model.add(LSTM(212, return_sequences=True,
               input_shape=(timesteps, data_dim)))
# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works
# add the final dense layer and then softmax
model.add(Dense(212, activation='softmax'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [19]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [20]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 5)
print("finished training!")

Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 1.5827 - acc: 0.7170
Epoch 2/5
1/1 [==============================] - 2s 2s/step - loss: 1.2857 - acc: 0.7170
Epoch 3/5
1/1 [==============================] - 2s 2s/step - loss: 2.4842 - acc: 0.7170
Epoch 4/5
1/1 [==============================] - 2s 2s/step - loss: 3.4182 - acc: 0.7217
Epoch 5/5
1/1 [==============================] - 2s 2s/step - loss: 3.2606 - acc: 0.7170
finished training!


In [22]:
# trying to view the model output
model.predict(X_train_data)

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e